**Inclass: Intro to Machine Learning 2**

- Data Analytics Specialization
- Course Length: 9 hours


## Training Objectives

- **Working with Time Series**
  - Data Preprocessing
  - Visualization: Multiple vs Multivariate Time Series
  
- **Modeling using `prophet`**
  - Baseline Model
  - Trend Component
  - Seasonality Component
  - Holiday Effects
  - Adding Regressors
  
- **Forecasting Evaluation**
  - Train-Test Split
  - Evaluation Metrics: MAPE
  - Expanding Window Cross Validation
  
- **Hyperparameter Tuning**

## Introduction

### Regression vs Time Series

Regresi dan time series sama-sama merupakan model machine learning yang target variabelnya berupa variabel dengan nilai numerik. Perbedaan dari kedua model ini adalah variabel yang digunakan untuk melakukan prediksi (atau pada kasus machine learning biasanya disebut sebagai prediktor). 

Kasus regresi menggunakan variabel-variabel dependent untuk memprediksi variabel independent (Contoh: menebak harga rumah dengan informasi jumlah kamar, luas bangunan, luas tanah). Sementara kasus time series berusaha menebak nilai di masa depan dengan mempelajari **pola dari variabel target** dari data historiknya. 

In [1]:
import pandas as pd

# pandas output display setup
pd.options.display.float_format = '{:,.4f}'.format

Pada course ini, kita akan menggunakan data dari perusahaan software Rusia - 1C Company yang tersedia pada [Kaggle platform](https://www.kaggle.com/c/competitive-data-science-predict-future-sales/overview). Data ini memiliki kondisi musiman atau seasonal serta beberapa 'noise' yang menunjukkan adanya suatu kejadian tertentu.

In [2]:
sales = pd.read_csv('data_input/sales_train.csv')
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.0000,1.0000
1,03.01.2013,0,25,2552,899.0000,1.0000
2,05.01.2013,0,25,2552,899.0000,-1.0000
3,06.01.2013,0,25,2554,"1,709.0500",1.0000
4,15.01.2013,0,25,2555,"1,099.0000",1.0000


Beberapa informasi dasar yang kita dapatkan yaitu:

- Terdapat 2,935,849 observasi/baris dan 6 variabel/kolom.
- Glossary data yang disediakan oleh Kaggle:
  - `date` adalah data tanggal dengan format **dd.mm.yyyy**
  - `date_block_num` adalah urutan angka untuk bulan (Jan 2013 = 0, Feb 2013 = 1, ..., Okt 2015 = 33)
  - `shop_id` adalah nomor identitas toko
  - `item_id` adalah nomor identitas produk
  - `item_price` adalah harga barang di tanggal tersebut
  - `item_cnt_day` adalah jumlah barang terjual di tanggal tersebut

Variabel yang akan kita coba untuk prediksi adalah **`item_cnt_day` dan `item_price`**. Kita akan melakukan analisis terkait kebutuhan pasar berdasarkan histori penjualan.

## Working with Time Series

Data time series didefinisikan sebagai data yang dikumpulkan dengan jarak waktu yang teratur (memiliki periode pengambilan data yang sama). Pada kasus ini, kita memiliki data penjualan software **harian**.
Sedangkan forecasting merupakan proses prediksi untuk data time series berdasarkan pola data historiknya.

### Data Preparation

Beberapa hal yang harus diperhatikan ketika melakukan analisis data time series yaitu:
- Melakukan konversi tipe kolom **date** ke `datetime64[ns]`
- Memastikan data sudah terurut dengan baik
- Memastikan tidak terdapat tanggal yang terlompat dan data kosong

Setelah memastikan seluruh hal di atas sudah terpenuhi, kita akan melakukan feature engineering dengan membuat kolom `total_revenue` yang nantinya akan diprediksi. Kolom ini merupakan hasil perhitungan dari `item_cnt_day` dengan `item_price`.

In [3]:
sales['date'] = pd.to_datetime(sales['date'], dayfirst=True)
sales.sort_values('date', inplace = True)
sales['total_revenue'] = sales['item_cnt_day'] * sales['item_price']

Ternyata data kita memiliki range waktu dari 1 Januari 2013 sampai 31 Oktober 2015.
Tetapi, data kita sebenarnya tersusun bukan dari satu toko dan produk saja, melainkan dari banyak toko dan banyak produk. Oleh karena itu kita akan melakukan analisis pada 3 toko paling populer berdasarkan jumlah transaksinya.
Mari kita buat tabel frekuensi untuk melihat toko paling populer ini.

In [4]:
top_3_shop = sales['shop_id'].value_counts().head(3).index
sales_top_3_shop = sales[sales['shop_id'].isin(top_3_shop)]

Dari tabel frekuensi di atas kita dapat melihat bahwa **toko 31, 25, dan 54** adalah toko dengan jumlah penjualan terbanyak. Kita akan berfokus pada tiga toko ini dengan melakukan filter pada data `sales`.

Hal paling penting dari sebuah data time series adalah datanya diambil pada interval waktu yang tetap dan jika kita lihat pada data di atas, terdapat beberapa data pada satu hari. Oleh karena itu kita bisa katakan bahwa data kita belum memenuhi syarat data time series. 
Pada data tiga toko terpopuler, masih tersimpan transaksi dari berbagai macam produk tiap harinya oleh karena itu proses yang perlu kita lakukan selanjutnya adalah melakukan **agregasi data** dimana kita akan menghitung jumlah benda yang terjual serta total revenue terhadap tiap toko dan tanggal.

In [5]:
daily_sales = sales_top_3_shop.groupby(['date', 'shop_id']).sum()[['item_cnt_day', 'total_revenue']].reset_index()
daily_sales.rename(columns = {'item_cnt_day' : 'total_qty'}, inplace=True)
daily_sales.head()

,date,shop_id,total_qty,total_revenue
0,2013-01-01,54,415.0000,"316,557.0000"
1,2013-01-02,25,568.0000,"345,174.1300"
2,2013-01-02,31,568.0000,"396,376.1000"
3,2013-01-02,54,709.0000,"519,336.0000"
4,2013-01-03,25,375.0000,"249,421.0000"


Sebagai catatan ketika melakukan agregasi data, kita hanya dapat mengubah bentuk data dari waktu yang jarak antar periodenya lebih singkat ke yang lebih panjang, sebagai contoh:

- periode **jam** ke **harian**
- periode **harian** ke **mingguan**
- periode **harian** ke **bulanan**
- periode **bulanan** ke **kuartal**
- periode **bulanan** ke **tahunan**, dst.

### Visualization: Multiple vs Multivariate Time Series

Sebelum kita masuk ke tahapan pembuatan model, kita perlu melakukan analisis berdasarkan visualisasi datanya. Pada bagian ini kita akan menggunakan library **`matplotlib` dan `seaborn`**. 

⚠️ Sedikit informasi tambahan tentang perbedaan *multiple* dengan *multivariate* pada kasus time series, berikut adalah definisi dari keduanya:

- Multiple time series: Terdapat **satu variabel** dari **banyak objek** yang ditinjau dari waktu ke waktu.

- Multivariate time series: Terdapat **banyak variabel** dari hanya **satu objek** yang ditinjau dari waktu ke waktu. Biasanya untuk kondisi ini variabelnya saling berkaitan satu sama lain.

In [ ]:
# data visualization library
import matplotlib.pyplot as plt
import seaborn as sns

#### Multiple Time Series

Pada kasus kita, multiple time series yaitu ketika kita meninjau fluktuasi dari variabel `total_qty` pada tiga toko terpopuler.

In [ ]:
sns.relplot(data=daily_sales, kind='line',
            x='date', y='total_qty', col='shop_id', aspect=1.5)
plt.show()

Dari visualisasi di atas, kita dapat melihat bahwa fluktuasi dari `total_qty` tidaklah sama, tetapi jika kita lihat lebih detail terdapat peningkatan yang sangat ekstrim pada `shop_id` 25 dan 31 pada akhir tahun, sedangkan pada toko 54 pola kenaikan di akhir tahunnya tidak sesignifikan kedua toko lain.

#### Multivariate Time Series

Pada kasus kita, multivariate time series adalah ketika kita meninjau fluktuasi dari `total_qty` dan `total_revenue` yang berubah terhadap waktu dari satu toko 31 saja. Kita akan melakukan conditional subsetting dari data `daily_sales` untuk membuat `daily_sales_31`.

In [ ]:
daily_sales_31 = daily_sales[daily_sales['shop_id'] == 31]

Create visualization :

In [ ]:
daily_sales_31.set_index('date')[['total_qty', 'total_revenue']].plot(subplots=True,figsize=(10, 5))
plt.suptitle('DAILY SOFTWARE SALES: STORE 31')
plt.show()

Dari visualisasi tersebut kita dapat melihat adanya fluktuasi yang serupa antara `total_qty` dan `total_revenue` di toko 31. Secara perspektif bisnis pun kedua variabel ini memiliki keterkaitan yang erat satu sama lain. Logikanya jika jumlah barang terjual meningkat maka revenue yang dihasilkan juga akan meningkat.

## Modeling using [`prophet`](https://facebook.github.io/prophet/)

Sebuah konsep dasar dalam pemahaman kasus time series adalah dengan melakukan pemecahan komponen dari sebuah data time series (**decompose**). Pemecahan komponen data time series ini banyak macamnya, dan salah satunya adalah menggunakan metode **General Additive Model (GAM)** yang menggambarkan sebuah data time series disusun dari penjumlahan berbagai komponen data. Sebagai permulaan, kita akan menggunakan 3 komponen:

- Trend ($T$): Pergerakan data dalam jangka panjang
- Seasonality ($S$): Perulangan akibat kondisi musiman
- Residuals ($E$): Komponen random yang tidak dapat dideskripsikan oleh trend maupun seasonal

Ide di balik GAM adalah ketiga komponen di atas dijumlahkan untuk menyusun data time series kita, matematisnya:

$Y(t) = T(t) + S(t) + E(t)$

Ketika kita membahas time series forecasting, terdapat sebuah sumsi yang perlu kita gunakan yaitu **Terdapat korelasi antara data/pengamatan yang berurutan**. Artinya peramalan/forecasting time series didasari oleh kondisi/perilaku di masa lalu. Jika disimpulkan maka konsepnya yaitu untuk melakukan peramalan nilai di masa depan, kita akan memperhatikan kondisi trend dan seasonal dari data time series.

Kelebihan dari Prophet yaitu adanya komponen tambahan yaitu **holiday effect** di luar komponen trend dan seasonal. Model ini akan memperhitungkan kejadian di hari-hari penting pada tanggal tertentu yang sudah terbukti berguna pada banyak kasus. Sebagai contoh adanya musim lebaran. Musim lebaran di Indonesia merupakan sesuatu yang sangat normal menimbulkan efek terhadap data time series, tetapi kondisi seasonal dari musim lebaran ini memiliki waktu yang berbeda-beda setiap tahunnya. Hal ini tidak dapat ditangkap oleh kondisi seasonal biasa tetapi dapat ditangkap dengan holiday effect pada Prophet.

### Baseline Model

#### Prepare the data

Untuk menggunakan `prophet`, kita perlu mempersiapkan data time series kita ke dalam format dataframe yang spesifik sesuai kebutuhan library ini. Dataframe ini membutuhkan 2 kolom:

- `ds`: kolom dengan informasi waktu, harus bertipe data `datetime64[ns]`
- `y`: nilai yang ingin kita forecast

Pada contoh di bawah kita akan menggunakan `total_qty` pada toko 31 sebagai nilai yang ingin kita forecast.

In [ ]:
daily_total_qty = daily_sales_31[['date', 'total_qty']].rename(
    columns = {'date':'ds', 
               'total_qty':'y'})

#### Fitting Model

Langkah selanjutnya adalah melakukan fitting model diawali dengan membuat `prophet` object menggunakan `Prophet()`. Setelah objectnya dibuat, kita akan melakukan fitting data `daily_total_qty`. Ide dari fitting adalah membiarkan mesin mencari informasi pola dari data time series yang nantinya akan digunakan untuk melakukan forecasting.

In [ ]:
# load the library
from prophet import Prophet
# fit model
model_31 = 

#### Forecasting

Berdasarkan data yang sudah ada, kita akan mencoba meramalkan data kita **1 tahun ke depan**. Untuk itu kita harus menyiapkan sebuah dataframe baru yang berisi informasi waktu yang ingin kita forecast. Hal ini dapat dilakukan menggunakan method `.make_future_dataframe()` dari `prophet`.

In [ ]:
future_31 = 

Dataframe `future_31` berisi informasi tanggal tepat 365 hari setelah data fitting berakhir. Kita akan menggunakan dataframe ini untuk melakukan forecasting menggunakan method `.predict()`.

In [ ]:
forecast_31 = 

Mari kita cek 5 hasil teratas dari `forecast_31`:

In [ ]:
## code here
forecast_31.head()

Dari berbagai nilai di atas kita akan mengambil beberapa informasi saja, yaitu:

In [ ]:
forecast_31[['ds', 'trend', 'weekly', 'yearly', 'yhat']]

Seperti yang sudah dibahas sebelumnya, pada General Additive Model (GAM) kita menggunakan asumsi bahwa data time series merupakan penjumlahan dari seluruh komponen. Pada kasus kita, dapat dilihat bahwa terdapat 3 buah komponen yaitu `trend`, seasonal `weekly`, dan seasonal `yearly`. Jika kita susun ketiga komponen ini dalam bentuk matematis maka didapatkan persamaan berikut:

$yhat(t) = T(t) + S_{weekly}(t) + S_{yearly}(t)$

<!-- Secara real, perhitungan sebenarnya pada data kita di atas adalah 
```python
df['yhat'] = df['trend'] * (1 + df['multiplicative_terms']) + df['additive_terms']
``` -->

#### Visualize

Setelah membuat model dan melakukan forecast, kita akan melakukan visualisasi dengan method `.plot()` dari `model_31` dengan inputnya adalah hasil forecastnya (`forecast_31`). Method ini akan menghasilkan objek `matplotlib` dengan informasi berupa data asli dalam bentuk scatter (titik-titik hitam) dan hasil model fitted serta forecast dengan garis biru.

In [ ]:
fig = model_31.plot(forecast_31)

Kita juga dapat membuat visualisasi komponen trend dan seasonalnya menggunakan method `.plot_components()`.

In [ ]:
fig = model_31.plot_components(forecast_31)

### Trend Component

Trend, yang merupakan pergerakan data secara general dalam jangka panjang ditampilkan secara menurun pada data kita. Metode yang diimplementasikan oleh Prophet secara default adalah perumusan **model linear** yang digambarkan sebagai berikut:

In [ ]:
#@title
# for illustration purposes only
from datetime import date

## prepare data
daily_sales_31_copy = daily_sales_31.copy()
daily_sales_31_copy['date_ordinal'] = daily_sales_31_copy['date'].apply(lambda date: date.toordinal())

## visualize regression line
plt.figure(figsize=(10, 5))
ax = sns.regplot(x='date_ordinal', y='total_qty', data=daily_sales_31_copy,
            scatter_kws={'color': 'black', 's': 2},
            line_kws={'color': 'red'})
new_labels = [date.fromordinal(int(item)) for item in ax.get_xticks()]
ax.set_xticklabels(new_labels)
plt.xlabel('date')
plt.show()

#### Automatic Changepoint Detection

Kelebihan dari `Prophet` yaitu dengan adanya *changepoint detection* yang berusaha mendeteksi secara otomatis adanya perubahan signifikan pada slope. Prophet akan membagi data kita pada beberapa titik dan menghitung slope per bagian secara terpisah.

Jumlah dari titik yang digunakan didefinisikan oleh parameter `n_changepoints` ketika membuat object `Prophet()` yang diletakkan secara merata pada 80% data awal (`changepoint_range = 0.8`).

Changepoint yang dibuat dapat dilihat dengan menggunakan fungsi `add_changepoints_to_plot` dan memasukkan beberapa parameter yang dapat dilihat pada docstring fungsi ini.

In [ ]:
#@title
# for illustration purposes only, threshold = 0
from prophet.plot import add_changepoints_to_plot
fig = model_31.plot(forecast_31)
a = add_changepoints_to_plot(fig.gca(), 
                             model_31, 
                             forecast_31, 
                             threshold=0) # seberapa besar minimal terjadinya perubahan slope

Dari 25 changepoints, prophet akan menghitung besarnya perubahan slope/kemiringan dan menentukan bagian yang berubah secara signifikan (nilai default thresholdnya adalah 0.01). 

In [ ]:
fig = model_31.plot(forecast_31)
a = add_changepoints_to_plot(fig.gca(), model_31, forecast_31)

Pada kasus kita, model mendeteksi **3 titik changepoint yang signifikan** dan memisahkan trend menjadi **4 bagian dengan slope yang berbeda**.

#### Adjusting Trend Flexibility

Prophet menyediakan parameter tuning untuk menyesuaikan fleksibilitas dari *changepoint detection*, berikut beberapa parameternya:

- `n_changepoints` (default = 25): Jumlah changepoint potensial, **tidak direkomendasikan** untuk diubah dan sebaiknya yang disesuaikan adalah parameter `changepoint_prior_scale`.
- `changepoint_range` (default = 0.8): Proporsi dari data historik yang akan diperhitungkan pada penentuan changepoint. Nilai yang direkomendasikan: [0.8, 0.95]
- `changepoint_prior_scale` (default = 0.05): Fleksibilitas dari trend, terutama seberapa besar perubahan dari trend ini di setiap changepoint. Nilai yang direkomendasikan: [0.001, 0.5]

💡 Peningkatan pada nilai default di setiap parameter di atas akan memberikan fleksibilitas pada garis trend (dan dapat menyebabkan overfitting pada training data). Sebaliknya, jika nilainya diturunkan maka trend akan jadi kurang fleksibel (dan malah menjadi underfitting)

In [ ]:
# define changepoint_range and changepoint_prior_scale
model_tuning_trend = Prophet(
    
    )

In [ ]:
# fitting model
______.fit(____)

In [ ]:
# make time window to predict
future = ____

# predict future dataframe
forecast = _____

In [ ]:
# visualize


### Seasonality Component

Selanjutnya kita akan membahas komponen time series lain yaitu seasonal. Berikut kita tinjau kembali plot masing-masing komponen pada data di toko 31.

In [ ]:
fig = model_31.plot_components(forecast_31)

Secara default, Prophet akan menentukan kondisi seasonal berdasarkan periodikal data yang disediakan. Pada kasus kita, data tersusun secara harian dari awal 2013 sampai akhir quarter 3 tahun 2015. 

- Setiap data dengan periode harian akan memiliki **weekly seasonality**.
- Sedangkan **yearly seasonality** secara default disetting sebagai `True` jika terdapat data lebih dari 2 tahun observasi harian.
- Komponen seasonal lain yang tidak terdeteksi pada kasus kita adalah **daily seasonality** yang berusaha mencari pola data dengan periode jam. Karena data kita tidak memiliki informasi jam, secara default nilai dari parameter ini adalah `False`.

#### Fourier Order

Prophet menggunakan deret Fourier untuk memperkirakan efek seasonal, metode ini bekerja dengan membuat fungsi periodik yang berisi **penjumlahan dari fungsi sin dan cos** (yang jumlahnya tidak terbatas).

💡 Fourier order (atau banyak penjumlahan parsial di atas) dapat ditentukan menggunakan sebuah parameter, parameter ini akan menetapkan seberapa cepat seasonality berubah. Dengan meningkatkan order, kita dapat membuat seasonality yang dihasilkan menjadi lebih fleksibel (dan mungkin menyebabkan model menjadi overfit), begitu pula sebaliknya. Berikut gambaran order:

<img src=https://funsubstance.com/uploads/gif/382/382951.gif>

Secara default, order dari `weekly_seasonality` adalah 3 dan `yearly_seasonality` adalah 10.

Kita akan coba lihat pengaruhnya dengan mengatur kedua parameter di atas ketika membuat object.

In [ ]:
# create object
model_tuning_seasonality = Prophet(
    
    )

# model fitting


# forecasting
future = 
forecast = 

# visualize
fig = 

#### Custom Seasonalities

Seasonality default yang disediakan oleh Prophet model untuk data harian adalah `weekly` dan `yearly`. Tetapi, bayangkan jika terdapat kasus berikut:

> Penjualan di bisnis Anda sangat dipengaruhi oleh tanggal bulanan. Kebanyakan dari customer cenderung membeli produk berdasarkan waktu pembayaran gaji mereka. Karena hal ini tidak ditangkap pada kondisi seasonal weekly dan yearly, kita perlu mendefinisikan seasonal non-regular. 

Terdapat dua langkah untuk tahapan ini:
1. Menghilangkan efek seasonal default (contoh: `yearly_seasonality = False`) jika dibutuhkan.
2. Menambahkan seasonality non-reguler menggunakan method `.add_seasonality()` setelah membuat object Prophet dan sebelum melakukan fitting.

Sehingga formula yang dibentuk menjadi:
$yhat(t) = T(t) + S_{weekly}(t) + \bf{S_{monthly}(t)}$

In [ ]:
# fitting model
model_custom_seasonality = Prophet(
    
    )

## add seasonality
model_custom_seasonality.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# fitting model


# forecasting
future = 
forecast = 

# visualize
fig = 

Untuk seasonal monthly, kita menggunakan periode 30.5 yang merupakan frekuensi non-reguler dalam satu musim. Nilai 30.5 ini adalah frekuensi yang umum digunakan pada monthly seasonality karena terdapat bulan dengan jumlah hari 30 dan 31 (dan 28/29 untuk Februari)

💡 Nilai Fourier order yang direkomendasikan berdasarkan seasonalnya:
- weekly seasonality = 3
- monthly seasonality = 5
- yearly seasonality = 10

### Holiday Effects

Salah satu keuntungan dari penggunaan Prophet adalah kemampuannya untuk memperhitungkan efek liburan. Holiday effect ini didefinisikan sebagai efek non-reguler yang disiapkan secara manual oleh user.

#### Modeling Holidays and Special Events

Sekarang kita akan melihat data kita dari model pertama yang kita buat. Dari visualisasi kita dapat melihat adanya penjualan yang signifikan di setiap akhir tahun (mencapai lebih dari 800 penjualan per hari).

In [ ]:
#@title
# for illustration purposes only
fig = model_31.plot(forecast_31)
plt.axhline(y=800, color='red', ls='--')
plt.show()

Mari kita lihat penjualan dengan jumlah di atas 800 menggunakan conditional subsetting.

In [ ]:
daily_total_qty[daily_total_qty['y'] > 800]

Ternyata penjualan yang sangat tinggi ini terjadi di tanggal **27 - 31 Desember** pada 2 tahun data kita. Dengan ini kita dapat mengasumsikan adanya fenomena tahun baru yaitu saat dimana orang-orang menggunakan bonus akhir tahun mereka untuk berbelanja.

Dari sini kita dapat mempersiapkan sebuah dataframe baru yang memiliki informasi di atas. Kolom-kolom yang harus dipersiapkan yaitu:

- `holiday`: nama hari libur yang nilainya unik
- `ds`: informasi waktu
- `lower_window` : jumlah periode waktu **sebelum** tanggal hari libur yang diasumsikan punya pengaruh (<= 0)
- `upper_window`: jumlah periode waktu **setelah** tanggal hari libur yang diasumsikan punya pengaruh (>= 0)

⚠️ Informasi ini harus mencakup seluruh kejadian yang akan kita gunakan, baik pada model fitting maupun forecast (mundur sejauh historikal data kita, dan maju sejauh kita ingin melakukan prediksi)

In [ ]:
holiday = pd.DataFrame({
    'holiday': 'new_year_eve',
    'ds': pd.to_datetime(['2013-12-31', '2014-12-31', # past date, historical data 
                          '2015-12-31']), # future date, to be forecasted
    'lower_window': -4, # include 27th - 31st December
    'upper_window': 0})
holiday

Setelah dataframe `holiday` dipersiapkan, kita perlu memasukkan informasi ini ke dalam `Prophet()` class pada parameter `holidays`.

In [ ]:
# fitting model
model_holiday = Prophet(
    
)
___.fit(___)

# forecasting
future = 
forecast = 

# visualize
fig = 

Dengan adanya informasi hari libur akhir tahun, terlihat bahwa model yang dihasilkan lebih baik menangkap pola di akhir tahun dibandingkan hanya menggunakan efek yearly seasonality. Jika kita plot komponennya, kita dapat melihat adanya efek komponen holiday yang terlampir:

In [ ]:
fig = model_holiday.plot_components(forecast)

### Adding Regressors

Informasi tambahan berupa regressor dapat diberikan sebagai bagian dari perhitungan linear. Hal ini bisa dilakukan dengan method `add_regressor()` sebelum model fitting. Pada kasus ini, kita akan menggunakan kolom `total_qty` sebagai regressor untuk meramal `total_revenue` sebagai tambahan dari komponen yang sudah ada sebelumnya (trend, seasonal, holiday). Oleh karena itu, secara matematis akan menjadi:

$revenue(t) = T_{revenue}(t) + S_{revenue}(t) + H_{revenue}(t) + \bf{qty(t)}$

⚠️ Sama seperti informasi hari libur, regressor tambahan ini harus disiapkan bukan hanya pada data historik tetapi juga pada data yang ingin kita ramal. Oleh karena itu variabel regressor ini haruslah sesuatu yang nilainya diketahui atau diprediksi secara terpisah menggunakan model time series juga (sehingga pemodelannya berlapis). Namun kita perlu **berhati-hati** jika menggunakan hasil prediksi karena **error dari hasil forecast regressor akan menghasilkan error pada target asli**.

In [ ]:
daily_sales_31.head()

#### Forecast the Regressor (`total_qty`)

Pada bagian ini, kita akan membuat model untuk meramal `total_qty` yang akan kita gunakan sebagai regressor untuk meramal `total_revenue`.

In [ ]:
# fitting model
model_total_qty = Prophet(
    n_changepoints=20, # trend flexibility
    changepoint_range=0.9, # trend flexibility
    changepoint_prior_scale=0.25, # trend flexibility
    weekly_seasonality=5, # seasonality fourier order
    yearly_seasonality=False, # remove seasonality
    holidays=holiday # new year eve effects
    )

## add monthly seasonality
model_total_qty.add_seasonality(name=___, period=___, fourier_order=5)
model_total_qty.fit(daily_total_qty)

# forecasting
future = model_total_qty.make_future_dataframe(periods=___, freq=___)
forecast_total_qty = model_total_qty.predict(future)

# visualize
fig = model_total_qty.plot(forecast_total_qty)

Dataframe di bawah ini adalah hasil peramalan total quantity untuk 365 hari paling akhir (dari 1 Nov 2015 sampai 30 Okt 2016).

In [ ]:
forecasted_total_qty = forecast_total_qty[['ds', 'yhat']].tail(365) \
                        .rename(columns={'yhat': 'total_qty'})
forecasted_total_qty.head()

Sementara, dataframe berikut ini adalah nilai asli/aktual dari total quantity yang kita gunakan untuk melakukan training model. Kita harus mengganti nama kolomnya agar sama dengan dataframe sebelumnya.

In [ ]:
actual_total_qty = daily_total_qty.rename(columns={'y': 'total_qty'})
actual_total_qty

Kedua dataframe di atas harus kita gabungkan agar informasi regressor `total_qty` dapat digunakan ketika meramalkan `total_revenue`. Nantinya dataframe yang tersusun haruslah terdiri dari:

- 1031 data awal dari nilai asli `total_qty`
- 365 data akhir dari nilai peramalan `total_qty`

In [ ]:
future_with_regressor = pd.concat([___, ___])
future_with_regressor

#### Forecast the Target Variable (`total_revenue`)

Setelah menyiapkan data di atas, kita akan membuat model Prophet untuk meramalkan `total_revenue` dan pada bagian ini kita akan menggunakan `total_qty` sebagai regressor. Karena kita akan mengambil kembali data dari `daily_sales_31` kita akan mengubah nama kolomnya menjadi `ds` untuk tanggal dan `y` untuk kolom target.

In [ ]:
daily_total_revenue = daily_sales_31[['date', 'total_revenue', 'total_qty']].rename(
    columns={'date': ___,
             'total_revenue': ___})

daily_total_revenue

Ketika melakukan fitting model dengan regressor, pastikan hal-hal berikut:
- Gunakan method `.add_regressor()` sebelum fitting model
- Lakukan peramalan menggunakan dataframe `future_with_regressor` yang sudah dipersiapkan, pastikan terdapat kolom dengan nama `ds` dan kolom regressornya.

In [ ]:
# fitting model
model_total_revenue = Prophet(
    ___ # new year eve effects
    )
## add regressor
model_total_revenue.add_regressor('total_qty')
model_total_revenue.fit(___)

# forecasting
## use dataframe with regressor, instead of just `ds` column
forecast_total_revenue = ___

# visualize
fig = ___

Jika kita plot komponennya, kita juga dapat melihat komponen tambahan yaitu regressor:

In [ ]:
fig = ___

## Forecasting Evaluation

Jika kita ingat kembali ketika melakukan analisis visual terhadap performa model peramalan kita sebelumnya. teknik yang kita lakukan tersebut sebenarnya lazim digunakan pada langkah cross-validation model (Walaupun pada kasus kita tidak terdapat data asli untuk dievaluasi). Secara sederhana, proses ini memisahkan data kita menjadi dua bagian:

- Train data digunakan untuk melatih model time series untuk mendapatkan pola seperti trend dan seasonal.
- Test data yang disimpan untuk tahap cross-validation yang berfungsi sebagai **unseen data** untuk melihat performa model.

Secara objektif proses ini dilakukan agar kita dapat melihat error model secara sekilas sehingga kita dapat menjadi bahan ekspektasi kita nantinya.

### Train-Test Split

Pada data ini, kita memiliki range waktu dari awal 2013 sampai q3 tahun 2015. Misalkan kita ingin menggunakan data di tahun 2015 sebagai test data dan sisanya untuk train data. Jika divisualisasikan maka titik-titik berwarna merah di bawah yang tidak akan difitting pada model Prophet kita.

In [ ]:
#@title
# for illustration purposes only
cutoff = pd.to_datetime('2015-01-01')
daily_total_qty['type'] = daily_total_qty['ds'].apply(
    lambda date: 'train' if date < cutoff else 'test')

plt.figure(figsize=(10, 5))
sns.scatterplot(x='ds', y='y', hue='type', s=7,
                palette=['black', 'red'],
                data=daily_total_qty)
plt.axvline(x=cutoff, color='gray', label='cutoff')
plt.legend()
plt.show()

Pemisahan kedua data seperti di atas dengan mudah dapat kita lakukan dengan conditional subsetting:

In [ ]:
train = daily_total_qty[___]
test = daily_total_qty[___]

print(f'Train size: {train.shape}')
print(f'Test size: {test.shape}')

Sekarang mari kita latih model kita menggunakan data tahun 2013-2014 dan forecast untuk tahun 2015 (**303 hari**).

In [ ]:
# fitting model
model_final = Prophet(
    ___, # holiday effect
    ___ # use yearly seasonality
) 
model_final.add_seasonality(___) # add monthly seasonality
model_final.fit(___) # only training set

In [ ]:
# forecasting
future_final = ___ # 303 days (test size)
forecast_final = ___

In [ ]:
# visualize
fig = model_final.plot(forecast_final)
plt.scatter(x=test['ds'], y=test['y'], s=10, color='red')
plt.show()

### Evaluation Metrics

Berdasarkan plot di atas model kita mampu untuk meramalkan kondisi di masa depan secara general, tetapi pada beberapa bagian masih berbeda antara data asli dengan data prediksi. Perbedaan kedua nilai itu disebut dengan error dan kita harus menghitung nilainya untuk mendapatkan kesimpulan evaluasi model. Berikut ini adalah beberapa metrics yang dapat digunakan:

- Root Mean Squared Error

<center>
$RMSE = \displaystyle{\sqrt{\frac{1}{n} \sum_{i=1}^{n}{(p_i - a_i)^2}}}$
</center>

- Mean Squared Error
<center>
$SSE = \Sigma{(y-\hat y)^2}$  
</center>

- Mean Precentage Absoulute Error
<center>
$MAPE = \frac{1}{n}\Sigma{\frac{| y-\hat y|}{y}} \times 100\%$  
</center>

Metrics-metrics di atas lazim digunakan untuk melakukan evaluasi model time series. Kita dapat memilih salah satu dari metricsnya dengan memahami nilai yang dihitung dan pastikan menggunakan nilai yang sama untuk satu kasus jika kita membuat beberapa model yang berbeda. Berikut ini adalah rangkuman dari 3 metrics di atas:

- RMSE (Root Mean Squared Error), metric yang paling umum digunakan untuk menghitung error numerik. Perhitungannya yaitu merata-ratakan hasil penjumlahan kuadrat dari error.
- SSE (Sum Squared Error), merupakan metric yang paling cepat komputasinya dan baik digunakan untuk benchmarking error model, tetapi hasilnya tidak dapat diinterpretasikan (karena perhitungannya merupakan hasil penjumlahan dari kuadrat error)
- MAPE (Mean Absolute Percentage Error), adalah metric yang paling sering digunakan pada model time series. Metric ini menghasilkan error dalam bentuk persentase berdasarkan range nilai aslinya. Kekurangannya yaitu tidak dapat digunakan ketika ada nilai 0 pada data kita (karena perhitungannya menggunakan nilai aktual pada faktor pembilang).

Pada course ini kita akan menggunakan MAPE karena paling mudah diinterpretasikan serta tidak ada nilai 0 pada data kita (dengan catatan tidak melakukan padding di atas, jika padding dilakukan dan nilai NaN diisi dengan 0 maka metric ini tidak dapat digunakan).

Untuk membantu perhitungan MAPE, kita akan menggunakan modul `metrics` dari library `sklearn`. Kita akan menghitung nilai error berdasarkan nilai `y` pada data aktual dan `yhat` pada data hasil forecast.

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

# get fitting result
forecast_train = forecast_final[forecast_final['ds'] < cutoff]

# compare with train data
train_mape = mean_absolute_percentage_error(y_true=train['y'],
                                     y_pred=forecast_train['yhat'])
train_mape

In [ ]:
# get forecast result
forecast_test = forecast_final[___]

# compare with test data
test_mape = mean_absolute_percentage_error(___,
                                     ___)
test_mape

### Expanding Window Cross Validation

Selain itu, kita bisa melakukan pengembangan lagi dengan melakukan lebih dari satu kali train-test split seperti gambar di bawah:

<center>
<img src="assets/forward-chaining-cv.png" width="600">
</center>

Prosedur ini disebut dengan **expanding window** dan dapat otomatis dilakukan dengan mengatur parameter pada method `cross_validation()`. Berikut ini adalah tiga parameter yang harus ditentukan:

- `initial`: Panjang waktu data training
- `horizon`: Panjang waktu forecast
- `period`: Jarak tanggal tiap cutoff

In [ ]:
from prophet.diagnostics import cross_validation
df_cv = cross_validation(model_holiday, 
                         initial='731 days', 
                         horizon='30 days', 
                         period='90 days')
df_cv

Proses cross-validation di atas akan dilakukan sebanyak 4 kali (4 folds), dimana setiap fold akan meramal 30 hari (`horizon`) sejak tanggal cutoff. Berikut adalah ilustrasi dari setiap fold:

In [ ]:
# for illustration purposes only
from helper import viz

df_copy = daily_total_qty[['ds', 'y']].copy()
df_cutoff_horizon = df_cv.groupby('cutoff')[['ds']].max()

viz.forward_chaining_viz(df_copy, df_cutoff_horizon)

Metric error setiap fold pada cross-validation dapat dihitung manual, contohnya MAPE di bawah ini:

In [ ]:
cv_mape = df_cv.groupby('cutoff').apply(
    lambda x: mean_absolute_percentage_error(y_true=x['y'],
                                     y_pred=x['yhat']))
cv_mape

Karena bisa dikatakan kita memiliki 4 model, kita dapat menghitung kembali **MAPE** model secara keseluruhan dengan mengambil rata-ratanya.

In [ ]:
cv_mape.mean()

## Hyperparameter Tuning

Hyperparameter tuning adalah proses pencarian kombinasi hyperparameter yang menghasilkan error terkecil. Pada kasus time series, hyperparameter yang diubah dapat berupa jumlah changepoint, fleksibilitas regularization, periode seasonal, dsb. 

Terdapat beerapa cara untuk melakukan tuning hyperparameter, salah satunya adalah **Grid Search Algorithm**. Algoritma ini akan melakukan pencarian untuk seluruh kombinasi dari opsi parameter yang kita sediakan. Karena sifatnya yang brute-force ini maka prosesnya dapat memakan waktu yang lama dan berat komputasinya, tetapi proses ini akan mendapatkan hyperparameter yang optimal dari nilai yang tersedia.

Pada bagian ini kita akan menggunakan Algoritma Grid Search untuk mencari hyperparameter terbaik pada model kita. Proses pencariannya akan dibantu oleh iterasi for-loop dengan sebelumnya kita tentukan dulu nilai-nilai hyperparameter yang ingin dicoba. Pada kasus time series kali ini kita akan mengevaluasi performa model menggunakan MAPE. Pada akhir prosesnya nanti kita akan mendapatkan kombinasi hyperparameter optimal yang menghasilkan MAPE terkecil. 


Silakan buka [link ini](https://facebook.github.io/prophet/docs/diagnostics.html#hyperparameter-tuning) untuk melihat rekomendasi hyperparameter yang dapat di-tuning pada model Prophet.

Mari kita coba pada model kita, langkah-langkahnya yaitu:

1. Siapkan parameter untuk dituning

In [ ]:
from tqdm import tqdm
import itertools

In [ ]:
# input grid search parameters
grid_params = {
    'changepoint_prior_scale': [0.001, 0.01, 0.05, 0.1],
    'changepoint_range': [0.8, 0.95]
}

# make all combination of parameters 
all_params = [dict(zip(grid_params.keys(), v)) for v in itertools.product(*grid_params.values())]
all_params

Kombinasi yang ingin kita tuning terdapat pada variabel `grid_params` yang menampung nilai-nilai untuk parameter changepoint_prior_scale dan changepoint_range. Library itertools digunakan untuk menyiapkan seluruh kombinasi yang mungkin dibuat dari opsi nilai yang kita sediakan dan hasilnya disimpan ke dalam variabel `all_params`.
Setelah parameter tersedia, kita akan melakukan fitting model untuk setiap kombinasi yang tersedia. 

In [ ]:
# Store the MAPE for each params here
mape = []  

# Use cross validation to evaluate all parameters
for params in tqdm(all_params):
    # fitting model
    # (TO DO: change the data and add other components: seasonality, holiday, regressor)
    model = Prophet(**params,
                  holidays=___)
    model.fit(___)

    # Expanding window cross validation (TO DO: use different values)
    cv = cross_validation(___)

    # Evaluation metrics: MAPE
    mape_s = cv.groupby('cutoff').apply(
      lambda x: mean_absolute_percentage_error(y_true=x['y'],
                                     y_pred=x['yhat']))

    mean_mape = mape_s.mean()
    mape.append(mean_mape)

Rangkaian kode di atas akan menjalankan model untuk setiap kombinasi hyperparameter pada model time series kita. Model tersebut dilatih menggunakan dataframe `daily_total_qty` dan dimasukkan ke dalam proses cross-validation dengan periode training selama **730 hari** dan horizon **30 hari**. Terakhir, model ini dievaluasi menggunakan metric MAPE yang disimpan ke dalam variabel `mape` untuk setiap kombinasi parameter yang digunakan.

❗ Library `tqdm` digunakan untuk melihat progress iterasi.

Dari variabel `all_params` dan `mape` kita dapat menyusun dataframe yang menyimpan kedua informasinya lalu mengurutkannya dari yang errornya paling kecil.

Dengan informasi parameter terbaik ini, kita lakukan kembali fitting model yang akan digunakan untuk forecasting.

## Dive Deeper

Pada latihan sebelumnya, kita membuat model time series menggunakan data `shop_id = 31`, coba ramalkan `total_revenue` menggunakan data `shop_id = 25`. Coba gunakan parameter-parameter untuk mengatur changepoint serta komponen seasonal maupun holiday. 

**Challenge**:
Coba bandingkan model tanpa regressor `total_qty` dengan tidak menggunakan regressor (jika menggunakan regressor berarti Anda harus membuat hasil prediksi regressornya terlebih dahulu).

## Optional Material

### [OPTIONAL - Matematika di Balik Trend]

Model linear pada trend akan membuat sebuah garis lurus melintasi sumbu x menggunakan metode ordinary least square (yaitu garisnya akan dibuat sedemikian rupa agar terdapat beda paling sedikit antara garis dengan nilai sesungguhnya dan menghasilkan perata-rataan yang berubah terhadap waktu). Konsepnya adalah membuat sebuah garis regresi linear dengan perumusan:

$$y=mx+C$$

Pada kasus kita, ketika kita ingin membuat model dari komponen trend maka `y` akan menjadi `Trend`, `m` akan menjadi perbedaan setiap perubahan waktu, dan C untuk intercept. Secara perumusan akan menjadi:

$$T(date)=m(date)+C$$

Untuk membuat ilustrasi slope pada trend, kita dapat membuat sebuah model dengan fungsi `ols()` dari library `statsmodels`:

In [ ]:
daily_sales_31_copy['date_ordinal']

In [ ]:
# import the library
import statsmodels.api as sm

# define predictor and target variable
X = daily_sales_31_copy['date_ordinal']
y = daily_sales_31_copy['total_qty']

# build ols model
X_train_sm = sm.add_constant(X)
lr = sm.OLS(y, X_train_sm).fit()

# printing the parameters
lr.params

Persamaan yang dibentuk berdasarkan hasil OLS di atas menjadi:

$$demand.in.qty=(−0.17)date+124413.733$$

Nilai (-0.17) adalah slope atau m pada persamaan yang dapat dibaca menjadi setiap kenaikan 1 tanggal (pada kasus ini berarti bertambah 1 hari), terjadi penurunan demand sebesar 0.17. Hal ini menunjukkan hubungan negatif antara waktu dengan demand.

### [OPTIONAL] Built-in Country Holidays

Kita dapat menggunakan informasi built-in tentang hari libur spesifik suatu negara menggunakan method `.add_country_holidays()` sebelum melakukan fitting model. Parameter yang harus ditambahkan yaitu `country_name` dan untuk Indonesia masukkan nilai `"ID"`

In [ ]:
model_holiday_indo = Prophet()
model_holiday_indo.add_country_holidays(country_name='ID')
model_holiday_indo.fit(daily_total_qty)

model_holiday_indo.train_holiday_names

💡 Sebagai tambahan, kita juga dapat secara manual menambahkan informasi hari libur di atas menggunakan modul `hdays` dari `prophet`. Cara ini biasanya digunakan ketika kita hanya ingin mengambil beberapa informasi hari libur, tidak semuanya.

Sebagai contoh kita ingin membuat list hari libur di Indonesia pada tahun 2020 dan 2021 secara manual.

- Fungsi `hdays.Indonesia()` digunakan untuk menginisiasi list hari libur di Indonesia
- Method `_populate()` digunakan untuk menambahkan list hari libur di tahun yang kita inginkan, hasilnya dalam bentuk dictionary dengan tanggal sebagai key dan nama hari libur sebagai value.

Langkah yang perlu dilakukan adalah membuatnya ke dalam bentuk dataframe. Kita akan menggunakan fungsi `pd.DataFrame()` dan merapikan bentuk dataframe kita agar sesuai dengan input yang diharapkan Prophet.

In [ ]:
# for illustration purposes only
# Reference: https://github.com/facebook/prophet/blob/master/python/prophet/hdays.py
from prophet import hdays
holidays_indo = hdays.Indonesia()
holidays_indo._populate(2020)
holidays_indo._populate(2021)
pd.DataFrame(holidays_indo, index = ['holiday']).T.rename_axis('ds').reset_index()